In [ ]:
#!pip -q install labelme 
#!pip -q install uuid
#!pip -q install albumentations 
#!pip -q install autotime
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import uuid
import cv2 
import utils
print("worked")

len(os.listdir('data/images/'))

## creating images Dataset


In [ ]:

def capture_imgs(img_num, path):
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    num = 0
        
    while(num < img_num):
        #*read img
        ret, frame = cap.read() 
        
        if cv2.waitKey(1) & 0xFF == ord('c'):
            print(f'collecting image {num}')
            #*create path for the read img
            img_name = os.path.join(path, f"{str(uuid.uuid1())}.jpg")
            #* writing the img
            cv2.imwrite(img_name, frame) 
            num +=1
            
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


## Labeling with labelme

In [ ]:
#!labelme

## Building Dataset and loader

In [ ]:
import tensorflow as tf
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt


## limiting GPU memory growth - avoid OOM

# gpus = tf.config.list_physical_devices('GPU')
# #gpus
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
    
tf.test.is_gpu_available()

### loading images into TF pipeline

In [ ]:
#* loading images as bytes and decoding them
def load_image(x):
    byte_img  = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img, channels=3)
    return img

images = tf.data.Dataset.list_files('data/images/*.jpg', shuffle=False)
#* testing
print(images.as_numpy_iterator().next(), len(list(images.as_numpy_iterator())))

#* using map to load images
images= images.map(load_image)
images = images.batch(4)
test_img = images.as_numpy_iterator().next()


fig = plt.subplots(figsize=(10, 10))
for i in range(test_img.shape[0]):
    plt.subplot(1, test_img.shape[0], i+1)
    plt.imshow(test_img[i])


### Splinting Dataset into train and test

In [ ]:
import pandas as pd


def retrieve_emotion(x):
    json_path = x.replace("jpg", "json").replace("images", "labels")
    if os.path.exists(json_path):
        with open(json_path) as f:
            data = json.load(f)
            emotion = data["shapes"][0]["label"]
            return emotion
    else:
        return "no face"


def retrieve_bbox(x):
    json_path = x.replace("jpg", "json").replace("images", "labels")
    if os.path.exists(json_path):
        with open(json_path) as f:
            data = json.load(f)
            bbox = data["shapes"][0]["points"]
            return bbox
    else:
        return [[0, 0], [0.0001, 0.00001]]


data_df = pd.DataFrame(os.listdir("data/images/"), columns=["images"])
data_df["images"] = data_df["images"].apply(lambda x: os.path.join("data/images/", x))
data_df["labels"] = data_df["images"].apply(retrieve_emotion)
data_df["bbox"] = data_df["images"].apply(retrieve_bbox)
data_df.to_csv("data/data.csv", index=False)
data_df

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df, _, _ = train_test_split(
    data_df,
    data_df["labels"],
    test_size=0.25,
    random_state=42,
    stratify=data_df["labels"],
    shuffle=True,
)
print(train_df.shape, val_df.shape)
train_df["labels"].value_counts() / len(train_df), val_df["labels"].value_counts() / len(val_df)

### Augmenting Dataset

In [ ]:
import albumentations as alb

augmentor = alb.Compose(
    [
        alb.RandomCrop(450, 450),
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
        alb.RandomBrightnessContrast(p=0.2),
        alb.RandomGamma(p=0.2),
        alb.RGBShift(p=0.2),
    ],
    bbox_params=alb.BboxParams(format="albumentations", label_fields=["class_labels"]),
)

In [ ]:
def augment_data(df:pd.DataFrame, split: str, num_copies = 120):
    aug_imgs_list = []
    aug_coords_list = []
    aug_label_list = []
    if not os.path.isdir(f'aug_data\{split}\images'):
        os.makedirs(f'aug_data\{split}\images')
    if (len(os.listdir(f'aug_data\{split}\images')) == 0):
        for item in range(len(df)):
            path = df.iloc[item]['images']
            img_name = path.split('/')[-1].split(".")[0]
            label = df.iloc[item]['labels']
            img = cv2.imread(path)
            coords = utils.cord_fixer(df.iloc[item]['bbox'], img)
            for x in range(num_copies):
                augmented = augmentor(image=img, bboxes=coords, class_labels=[label])
                cv2.imwrite(f'aug_data/{split}/images/{img_name}_{x}.jpg', augmented['image'])
                aug_imgs_list.append(f'aug_data/{split}/images/{img_name}_{x}.jpg')
                
                try:
                    aug_label_list.append(augmented['class_labels'][0])
                    aug_coords  = augmented['bboxes'][0]
                    #print(type(aug_coords))
                    aug_coords_list.append(aug_coords)
                except:
                    aug_label_list.append(label)
                    aug_coords_list.append((0,0,0,0))
                    
        aug_df = pd.DataFrame({'images': aug_imgs_list, 'labels': aug_label_list, 'bbox': aug_coords_list})
        aug_df.to_csv(f'aug_data/{split}/train_data.csv', index=False)
    else:
        print(f'{split} data already augmented')


augment_data(train_df, 'train', num_copies=125)
augment_data(val_df, 'val', num_copies=15)

aug_train_df = pd.read_csv('aug_data/train/train_data.csv')
aug_train_df['bbox'] = aug_train_df['bbox'].apply(lambda x: eval(x))

aug_val_df = pd.read_csv('aug_data/val/train_data.csv')
aug_val_df['bbox'] = aug_val_df['bbox'].apply(lambda x: eval(x))


aug_train_df

In [ ]:
aug_train_df['labels'].value_counts()

In [ ]:
sample = aug_train_df.iloc[1250]
img = cv2.imread(sample['images'])
coords = sample['bbox']
img_label = sample['labels']

cv2.rectangle(
    img,
    (
        int(coords[0] * img.shape[0]),
        int(coords[1] * img.shape[1]),
    ),
    (
        int(coords[2] * img.shape[0]),
        int(coords[3] * img.shape[1]),
    ),
    (0, 255, 0),
    2,
)
print(img_label)
plt.imshow(img)

In [ ]:

def show_labeled_img(path):
    label_path = path.replace('images', 'labels').replace('jpg', 'json')
    coords = json.load(open(label_path))['shapes'][0]['points']
    img = cv2.imread(path)
    coords = utils.cord_fixer(coords, img)
    cv2.rectangle(
        img = img,
        pt1=(int(coords[0][0] * img.shape[1]), int(coords[0][1] * img.shape[0])),
        pt2=(int(coords[0][2] * img.shape[1]), int(coords[0][3] * img.shape[0])),
        color=(0, 255, 0),
        thickness=2,
    )
    plt.imshow(img)

path = r'data\images\99ee658a-e836-11ed-89c2-60189528f842.jpg'
show_labeled_img(path)

## Create Augmented Dataset


In [ ]:
def load_image(x):
    byte_img  = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img, channels=3)

    return img

class_dict = {
    'no face' : 0,
    'happy' : 1,
    'sad' : 2,
    'natural' : 3,
    'surprised' : 4,
    'angry' : 5,
}

reverse_dict  = {v:k for k,v in class_dict.items()}

aug_train_df = pd.read_csv('aug_data/train/train_data.csv')

def load_dataset_from_df(df, class_dict = class_dict ):
    #aug_train_df['bbox'] = aug_train_df['bbox'].apply(lambda x: eval(x))
    df['bbox'] = df['bbox'].apply(lambda x:eval(x))
    df['xmin'] = df['bbox'].apply(lambda x: x[0]).astype('float32')
    df['ymin'] = df['bbox'].apply(lambda x: x[1]).astype('float32')
    df['xmax'] = df['bbox'].apply(lambda x: x[2]).astype('float32')
    df['ymax'] = df['bbox'].apply(lambda x: x[3]).astype('float32')
    df['labels'] = df['labels'].apply(lambda x: class_dict[x]).astype('float32')

    #* one hot encoding
    onehot_labels = tf.keras.utils.to_categorical(df['labels'], num_classes=6)
    onehot_labels = tf.convert_to_tensor(onehot_labels, dtype=tf.float32)

    dataset_label = tf.data.Dataset.from_tensor_slices(onehot_labels)
    # aug_train_df.drop('bbox', axis=1, inplace=True)
    dataset_images = tf.data.Dataset.from_tensor_slices(df[['images']])
    dataset_images = dataset_images.map(lambda x: load_image(x[0]))
    dataset_images = dataset_images.map(lambda x : tf.image.resize(x, (120, 120)), num_parallel_calls=tf.data.AUTOTUNE)
    dataset_images = dataset_images.map(lambda x : x/255, num_parallel_calls=tf.data.AUTOTUNE)

    data_set_coords = tf.data.Dataset.from_tensor_slices((df[['xmin', 'ymin', 'xmax', 'ymax']]))
    dataset = tf.data.Dataset.zip((dataset_images, dataset_label , data_set_coords))
    return dataset

dataset = load_dataset_from_df(aug_train_df)
#* for prediction
## reverse_dict[y[0].argmax()]


In [ ]:
def _create_augmented_data_deprecated(split, augmentor, n_img):
    for image in os.listdir(os.path.join('data', split, 'images')):
        img  = cv2.imread(os.path.join('data', split, 'images', image))
        coords = [0, 0 , 0.000001, 0.000001]
        label_path = os.path.join('data', split, 'labels', image.split('.')[0]+'.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label =json.load(f)
                coords = label['shapes'][0]['points']
                coords = utils.cord_fixer(coords, img)
                img_label  = label['shapes'][0]['label']
        try :
            for x in range (n_img):
                #* saving the augmented image
                augmented  = augmentor(image=img, bboxes =coords, class_labels = [img_label])
                cv2.imwrite(os.path.join('aug_data', split, 'images', image.split('.')[0]+f'_{x}.jpg'), augmented['image'])
                #* saving the augmented label
                annotation = {}
                annotation['image'] = image
                
                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0:
                        annotation['bbox'] = [0.0, 0.0, 0.0, 0.0]
                        annotation['class'] = 'no face'
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = augmented['class_labels'][0]
                else:
                    annotation['bbox'] = [0.0, 0.0, 0.0, 0.0]
                    annotation['class'] = 'no face'
                    
                with open(os.path.join('aug_data', split, 'labels', image.split('.')[0] + f'_{x}.json'), 'w') as f:
                    json.dump(annotation, f)
        except Exception as e :
            print(e)
                    


### Preparing Dataset for training

In [ ]:
def load_image(x):
    byte_img  = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img, channels=3)
    return img


def _prepare_data_deprecated(path):
    images = tf.data.Dataset.list_files(path, shuffle=False)
    images = images.map(utils.load_image, num_parallel_calls=tf.data.AUTOTUNE)
    images = images.map(lambda x : tf.image.resize(x, (120, 120)), num_parallel_calls=tf.data.AUTOTUNE)
    images = images.map(lambda x : x/255, num_parallel_calls=tf.data.AUTOTUNE)
    return images
    



## combining labels and images

In [ ]:
def _combine_deprecated(images, labels):
    data = tf.data.Dataset.zip((images, labels))
    data = data.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
    return data

# train = _combine(train_images, train_label)
# test = _combine(test_images, test_label)
# val = _combine(val_images, val_label)
# sample_iterator = train.as_numpy_iterator()

In [ ]:
sample = next(iter(dataset.batch(4)))
img = sample[0][0].numpy()
label = sample[1][0].numpy()
label = reverse_dict[label.argmax()]
coords = sample[2][0].numpy()

cv2.rectangle(
    img,
    (int(coords[0] * img.shape[0]), int(coords[1] * img.shape[1])),
    (int(coords[2] * img.shape[0]), int(coords[3] * img.shape[1])),
    (0, 255, 0),
    2,
)

plt.imshow(img)
plt.title(label)